**Phần 1:** Tạo ra file data - tfidf1 - chạy bằng file python

**Phần 2:** tfidf2 

In [1]:
from collections import defaultdict
from os import listdir
from os.path import isfile
from nltk.stem.porter import PorterStemmer
import re
import numpy as np

In [2]:
with open("/content/drive/MyDrive/Colab Notebooks/DSLAB TRAINING/SESSION 1/20news-full-processed.txt", encoding="utf8", errors='ignore') as f:
  lines = f.read().splitlines()
len(lines) 
# Have 18846 docs
# docs: (label, doc_id, text): 0<fff>53068<fff>decay cbnewsj cb att com dean kaflowitz subject

18846

In [3]:
# Compute idf
def compute_idf(df, corpus_size):
    assert df > 0
    return np.log10(corpus_size * 1. / df)

In [4]:
# Generate vocab (word, idf): proselyt<fff>3.2338265016239234
def generate_vocabulary(data_path):
    with open(data_path, encoding="utf8", errors='ignore') as f:
        lines = f.read().splitlines()
    doc_count = defaultdict(int)
    corpus_size = len(lines)

    for line in lines:
        features = line.split('<fff>')
        text = features[-1]
        words = list(set(text.split()))
        for word in words:
            doc_count[word] += 1

    words_idfs = [(word, compute_idf(document_freq, corpus_size))
                   for word, document_freq in zip(doc_count.keys(), doc_count.values())
                   if document_freq > 10 and not word.isdigit()]
    words_idfs.sort(key=lambda word_idf: -word_idf[1])
    print('Vocabulary size: {}'.format(len(words_idfs)))
    with open("/content/drive/MyDrive/Colab Notebooks/DSLAB TRAINING/SESSION 1/words_idfs.txt", 'w') as f:
        f.write('\n'.join([word + '<fff>' + str(idf) for word, idf in words_idfs]))

In [5]:
generate_vocabulary("/content/drive/MyDrive/Colab Notebooks/DSLAB TRAINING/SESSION 1/20news-full-processed.txt")
# Vocabulary size: 14079

Vocabulary size: 14079


In [6]:
def get_tf_idf(data_path):
  with open("/content/drive/MyDrive/Colab Notebooks/DSLAB TRAINING/SESSION 1/words_idfs.txt", encoding="utf8", errors='ignore') as f:
      words_idfs = [(line.split('<fff>')[0], float(line.split('<fff>')[1]))
                    for line in f.read().splitlines()]
      words_IDs = dict([(word, index)
                        for index, (word, idf) in enumerate(words_idfs)])
      idfs = dict(words_idfs)

  with open(data_path, encoding="utf8", errors='ignore') as f:
      documents = [(int(line.split('<fff>')[0]),
                    int(line.split('<fff>')[1]),
                    line.split('<fff>')[2])
                    for line in f.read().splitlines()]

      data_tf_idf = []
      for document in documents:
          label, doc_id, text = document
          words = [word for word in text.split() if word in idfs]
          word_set = list(set(words))
          max_term_freq = max([words.count(word) for word in word_set])
          words_tfidfs = []
          sum_squares = 0.0

          for word in word_set:
              term_freq = words.count(word)
              tf_idf_value = term_freq * (1. / max_term_freq) * idfs[word]
              words_tfidfs.append((words_IDs[word], tf_idf_value))
              sum_squares += tf_idf_value ** 2

          words_tfidfs_normalize = [str(index) + ':'
                                    + str(tf_idf_value / np.sqrt(sum_squares))
                                    for index, tf_idf_value in words_tfidfs]
          sparse_rep = ' '.join(words_tfidfs_normalize)
          data_tf_idf.append((label, doc_id, sparse_rep))
  return data_tf_idf

In [7]:
data_tf_idf = get_tf_idf("/content/drive/MyDrive/Colab Notebooks/DSLAB TRAINING/SESSION 1/20news-full-processed.txt")

In [8]:
data_tf_idf
# (label, doc_id, sparse_rep): (2, 10131, '14078:0.0 10864:0.1092775151817038 13253:0.078910118528671 ...')

Output hidden; open in https://colab.research.google.com to view.

In [9]:
len(data_tf_idf)
# docs

18846